In [15]:
import json
import requests
import pandas as pd

In [10]:
# load in api key
credentials = json.load(open('./apikey.json', 'r'), encoding='utf-8')

In [11]:
# GHE docs:
# curl -H "Authorization: token OAUTH-TOKEN" http(s)://[hostname]/api/v3  <-- endpoint
''' 
Declare global variables
'''
baseurl = 'git.generalassemb.ly' # of course may be different for you
header = {'Authorization': 'token {}'.format(credentials['token'])}
source_org = 'Data-science-immersive' # change this as you see fit
target_org = 'DSI-ME-1' # this one too
repo_names = []
data = { 'organization': target_org }

In [14]:
'''
Example calls below
'''
# call the base endpoint
'''
r = requests.get('https://{}/api/v3'.format(baseurl), headers=header)
'''

# get a listing of orgs the user's api key has access to

r = requests.get('https://{}/api/v3/user/orgs'.format(baseurl), headers=header)
orgs = json.loads(r.content)
for org in orgs:
    print('{}: id = {}'.format(org['login'], org['id']))

dsi-unit-3: id = 261
DSI-ATX-3: id = 6931
data-part-time: id = 6944
DSI-EAST-1: id = 7793
DSI-DC-6: id = 7895
DSI-EAST-2: id = 9038
Data-science-immersive: id = 9634
DAT-ME-1: id = 11079
DSI-ME-1: id = 11622


### Code below will write an entire org's repos to a local csv file

In [16]:
'''
NEEDS CODE WRITTEN TO TRAVERSE PAGES (start at page 1, go until r.status_code = 202)
see docs here:
https://developer.github.com/v3/repos/#parameters-2

Note: the 'page' parameter is not documented with the github API, I found it by trial/error
'''

# get a listing of all repos in the target org, 'Data-science-immersive'
# from GHE docs: GET /orgs/:org/repos

r = requests.get('https://{}/api/v3/orgs/{}/repos'.format(baseurl, source_org), \
                 headers=header, params={ 'type': 'all' , 'page': 1 })
repos = json.loads(r.content)

clones = []
for repo in repos:
    clones.append(repo['ssh_url'])
    
# show first 5
for repo in repos[:5]:
    print('{}, {}'.format(repo['name'], repo['ssh_url']))

templates, git@git.generalassemb.ly:Data-science-immersive/templates.git
python-101-lesson, git@git.generalassemb.ly:Data-science-immersive/python-101-lesson.git
python-comprehensions-lab, git@git.generalassemb.ly:Data-science-immersive/python-comprehensions-lab.git
python-comprehensions-lesson, git@git.generalassemb.ly:Data-science-immersive/python-comprehensions-lesson.git
python-functions-lesson, git@git.generalassemb.ly:Data-science-immersive/python-functions-lesson.git


In [81]:
# set initial page to fetch
page = 1
# begin fetchin those shits
while True:
    r = requests.get('https://{}/api/v3/orgs/{}/repos'.format(baseurl, source_org), \
             headers=header, params={ 'type': 'all' , 'page': page })
    try:
        r.json()[0]
        print('Successfully fetched page {}'.format(page))
        page += 1
    except IndexError:
        print('Page {} returned no data, exiting...'.format(page))
        break

Successfully fetched page 1
Successfully fetched page 2
Successfully fetched page 3
Successfully fetched page 4
Successfully fetched page 5
Page 6 returned no data, exiting...


In [48]:
r.json()

[]

In [30]:
df_dict = {
    'id': [], 
    'name': [],
    'full_name': [],
    'ssh_url': [],
    'created_at': [],
    'updated_at': [],
    'pushed_at': []
}
# create initial blank dataframe
df = pd.DataFrame.from_dict(df_dict).set_index('id')
for repo in repos:
    for k in df_dict.keys():
        df_dict[k].append(repo[k])
res_df = pd.DataFrame.from_dict(df_dict).set_index('id')
# clear dict
for k in df_dict.keys():
    df_dict[k] = []
# append res_df to initial blank dataframe
df.append(res_df)

,name,full_name,ssh_url,created_at,updated_at,pushed_at
id,,,,,,
122821.0,templates,Data-science-immersive/templates,git@git.generalassemb.ly:Data-science-immersiv...,2018-04-25T20:23:22Z,2018-04-25T20:27:03Z,2018-04-25T20:28:20Z
124362.0,python-101-lesson,Data-science-immersive/python-101-lesson,git@git.generalassemb.ly:Data-science-immersiv...,2018-04-30T18:08:08Z,2018-04-30T18:08:40Z,2018-04-30T18:08:39Z
124364.0,python-comprehensions-lab,Data-science-immersive/python-comprehensions-lab,git@git.generalassemb.ly:Data-science-immersiv...,2018-04-30T18:08:57Z,2018-04-30T18:09:21Z,2018-04-30T18:09:19Z
124376.0,python-comprehensions-lesson,Data-science-immersive/python-comprehensions-l...,git@git.generalassemb.ly:Data-science-immersiv...,2018-04-30T18:09:31Z,2018-04-30T18:09:49Z,2018-04-30T18:09:48Z
124379.0,python-functions-lesson,Data-science-immersive/python-functions-lesson,git@git.generalassemb.ly:Data-science-immersiv...,2018-04-30T18:10:32Z,2018-04-30T18:11:01Z,2018-04-30T18:11:00Z
124381.0,python-intro_to_python_pokemon-lab,Data-science-immersive/python-intro_to_python_...,git@git.generalassemb.ly:Data-science-immersiv...,2018-04-30T18:12:07Z,2018-04-30T18:12:24Z,2018-04-30T18:12:22Z
124384.0,1.02-lesson-python-control-flow,Data-science-immersive/1.02-lesson-python-cont...,git@git.generalassemb.ly:Data-science-immersiv...,2018-04-30T18:12:39Z,2018-07-25T18:26:40Z,2018-07-30T07:31:41Z
124528.0,statistics-central_limit_theorem-lesson,Data-science-immersive/statistics-central_limi...,git@git.generalassemb.ly:Data-science-immersiv...,2018-04-30T20:37:26Z,2018-04-30T20:37:43Z,2018-04-30T20:37:41Z
124529.0,statistics-continuous_distributions-lesson,Data-science-immersive/statistics-continuous_d...,git@git.generalassemb.ly:Data-science-immersiv...,2018-04-30T20:38:03Z,2018-04-30T20:38:34Z,2018-04-30T20:38:32Z


In [22]:
repos

[{'id': 122821,
  'name': 'templates',
  'full_name': 'Data-science-immersive/templates',
  'owner': {'login': 'Data-science-immersive',
   'id': 9634,
   'avatar_url': 'https://avatars.git.generalassemb.ly/u/9634?',
   'gravatar_id': '',
   'url': 'https://git.generalassemb.ly/api/v3/users/Data-science-immersive',
   'html_url': 'https://git.generalassemb.ly/Data-science-immersive',
   'followers_url': 'https://git.generalassemb.ly/api/v3/users/Data-science-immersive/followers',
   'following_url': 'https://git.generalassemb.ly/api/v3/users/Data-science-immersive/following{/other_user}',
   'gists_url': 'https://git.generalassemb.ly/api/v3/users/Data-science-immersive/gists{/gist_id}',
   'starred_url': 'https://git.generalassemb.ly/api/v3/users/Data-science-immersive/starred{/owner}{/repo}',
   'subscriptions_url': 'https://git.generalassemb.ly/api/v3/users/Data-science-immersive/subscriptions',
   'organizations_url': 'https://git.generalassemb.ly/api/v3/users/Data-science-immersive

### Code below will clone an entire org's repos to a local folder; use with parseme.ipynb

In [ ]:
'''
NEEDS CODE WRITTEN TO TRAVERSE PAGES (start at page 1, go until r.status_code = 202)
see docs here:
https://developer.github.com/v3/repos/#parameters-2

Note: the 'page' parameter is not documented with the github API, I found it by trial/error
'''

# get a listing of all repos in the target org, 'Data-science-immersive'
# from GHE docs: GET /orgs/:org/repos

r = requests.get('https://{}/api/v3/orgs/{}/repos'.format(baseurl, source_org), \
                 headers=header, params={ 'type': 'all' , 'page': 1 })
repos = json.loads(r.content)

clones = []
for repo in repos:
    clones.append(repo['ssh_url'])
    
# show first 5
for repo in repos[:5]:
    print('{}, {}'.format(repo['name'], repo['ssh_url']))

In [ ]:
len(clones)

In [ ]:
'''
This will create a place for all the cloned repos to live.
Use the parseme file, pointing to this directory, to
scan all clones repos for words
'''

%%bash
if [ ! -d "./course_dump" ]; then
    mkdir ./course_dump
    echo "created directory ./course_dump"
else
    echo "directory already exists: ./course_dump"
fi

In [ ]:
# WARNING!!! ===
# this will clone ALL repositories in the organization to ./course_dump!!
# note - has bug; will not retry cloning of repository if network fails :'(

try:
    for repo in clones:
        # made it sleepy to be a good citizen
        !cd ./course_dump && git clone $repo && sleep 1
    print("Finished cloning {} repositories into ./course_dump".format(len(clones)))
except:
    # outstanding error handling here
    print("noooooo!!")

### This will fork all repositories from a source org to a target org

In [ ]:
r = requests.get('https://{}/api/v3/orgs/{}/repos'.format(baseurl, source_org), \
                 headers=header)
repos = json.loads(r.content)

fork_urls = []
for repo in repos:
    fork_urls.append(repo['forks_url'])

In [ ]:
# set the target org parameter, where the forks will go
data = { 'organization': target_org }

'''
This needs to be done as a session because the GET request does the
auth, and the POST request does the forking. You CAN'T pass a cookie
between the two sessions, I tried. Couldn't find any examples of anybody
doing this on SO, etc. Was able to get it working with CURL but wanted
to use python requests lib. Trick is passing the header with BOTH the 
GET *AND* the POST requests. You can't just send the header with the 
POST request, despite what the docs lead you to believe.

School of hard knocks... damn it feels good to be a gangster
'''

with requests.Session() as s:
    # this get request authenticates the user
    r1 = s.get('https://{}/api/v3'.format(baseurl), headers=header)
    # ...and then this post request creates the fork
    for fork_url in fork_urls:
        r2 = s.post(fork_url, headers=header, data=json.dumps(data))
        print('{} successfully forked to {}'.format( \
            fork_url.split('/')[-3] + '/' + fork_url.split('/')[-2], \
            json.loads(r2.text)['full_name'])
        )